In [1]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Read in the CSV file and create the Pandas DataFrame
df_project4 = pd.read_csv(
    Path("C:/Users/PC2/Downloads/Starter_Data.csv")
)
# Review the DataFrame
df_project4.head()

,Date,Price_Index_Detail_Toronto_Monthly_All_Items,Price_Index_Change,Inflation Rate,Canadian_Prime_Rate,CND_TSX_Indx,Unemployment_Rate,GDP_growth,CND_USD_Exchange
0,2004-10-01,104.9,0.000,0.023,0.043,0.023,0.059,0.065,0.800
1,2004-11-01,105.2,0.003,0.024,0.043,0.018,0.059,0.065,0.835
2,2004-12-01,105.1,-0.001,0.021,0.043,0.024,0.059,0.065,0.824
3,2005-01-01,105.0,-0.001,0.019,0.043,-0.005,0.057,0.064,0.826
4,2005-02-01,105.6,0.006,0.021,0.043,0.050,0.057,0.064,0.806


In [4]:
# Remove white spaces
df_project4.columns = df_project4.columns.str.strip()
df_project4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 9 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Date                                          219 non-null    object 
 1   Price_Index_Detail_Toronto_Monthly_All_Items  219 non-null    float64
 2   Price_Index_Change                            219 non-null    float64
 3   Inflation Rate                                219 non-null    float64
 4   Canadian_Prime_Rate                           219 non-null    float64
 5   CND_TSX_Indx                                  219 non-null    float64
 6   Unemployment_Rate                             219 non-null    float64
 7   GDP_growth                                    219 non-null    float64
 8   CND_USD_Exchange                              219 non-null    float64
dtypes: float64(8), object(1)
memory usage: 15.5+ KB


In [6]:
column_to_scale = df_project4[["Price_Index_Detail_Toronto_Monthly_All_Items","Price_Index_Change",	"Inflation Rate","Canadian_Prime_Rate",	"CND_TSX_Indx",	"Unemployment_Rate","GDP_growth","CND_USD_Exchange"]]

In [8]:
# Applying Scaler
scaler = StandardScaler()
project4_scaled = scaler.fit_transform(column_to_scale)

In [9]:
project4_scaled

array([[-1.56427223, -0.44523807,  0.14959647, ..., -0.05118592,
         0.4937341 , -0.60419126],
       [-1.54176151,  0.28974035,  0.21756664, ..., -0.05118592,
         0.4937341 , -0.24375098],
       [-1.54926509, -0.69023087,  0.01365611, ..., -0.05118592,
         0.4937341 , -0.35703221],
       ...,
       [ 2.26254993,  1.02471877,  3.27622468, ..., -1.68593609,
         1.71201039, -1.19125581],
       [ 2.29256422,  0.28974035,  3.20825451, ..., -1.68593609,
         1.71201039, -1.32603988],
       [ 2.26254993, -1.18021649,  2.86840361, ..., -1.68593609,
         1.71201039, -1.22595076]])

In [10]:
# Creating a DataFrame with with the scaled data
df_project4_transformed = pd.DataFrame(project4_scaled, columns=["Price_Index_Detail_Toronto_Monthly_All_Items","Price_Index_Change",	"Inflation Rate","Canadian_Prime_Rate",	"CND_TSX_Indx",	"Unemployment_Rate","GDP_growth","CND_USD_Exchange"])
df_project4_transformed.head() 

,Price_Index_Detail_Toronto_Monthly_All_Items,Price_Index_Change,Inflation Rate,Canadian_Prime_Rate,CND_TSX_Indx,Unemployment_Rate,GDP_growth,CND_USD_Exchange
0,-1.564272,-0.445238,0.149596,0.651019,0.481037,-0.051186,0.493734,-0.604191
1,-1.541762,0.289740,0.217567,0.651019,0.351586,-0.051186,0.493734,-0.243751
2,-1.549265,-0.690231,0.013656,0.651019,0.506927,-0.051186,0.493734,-0.357032
3,-1.556769,-0.690231,-0.122284,0.651019,-0.243888,-0.284722,0.470748,-0.336436
4,-1.511747,1.024719,0.013656,0.651019,1.180071,-0.284722,0.470748,-0.542402


In [11]:
# Define features set X by selecting all columns but GDP_growth
X = df_project4_transformed.drop(columns=["GDP_growth"]).copy()

# Display the features DataFrame
X.head()

,Price_Index_Detail_Toronto_Monthly_All_Items,Price_Index_Change,Inflation Rate,Canadian_Prime_Rate,CND_TSX_Indx,Unemployment_Rate,CND_USD_Exchange
0,-1.564272,-0.445238,0.149596,0.651019,0.481037,-0.051186,-0.604191
1,-1.541762,0.289740,0.217567,0.651019,0.351586,-0.051186,-0.243751
2,-1.549265,-0.690231,0.013656,0.651019,0.506927,-0.051186,-0.357032
3,-1.556769,-0.690231,-0.122284,0.651019,-0.243888,-0.284722,-0.336436
4,-1.511747,1.024719,0.013656,0.651019,1.180071,-0.284722,-0.542402


In [12]:
# Define target set by selecting the GDP_growth column
y = df_project4_transformed["GDP_growth"]

# Display a sample of y
y[:5]

0    0.493734
1    0.493734
2    0.493734
3    0.470748
4    0.470748
Name: GDP_growth, dtype: float64

In [13]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [65]:
# Define the model - deep neural net
number_input_features = 7
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 4

nn = Sequential()
# First hidden layer
nn.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)
# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))
# Output layer
nn.add(Dense(units=1, activation="sigmoid"))

In [41]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 8)                 64        
                                                                 
 dense_19 (Dense)            (None, 4)                 36        
                                                                 
 dense_20 (Dense)            (None, 1)                 5         
                                                                 
Total params: 105 (420.00 Byte)
Trainable params: 105 (420.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
# Compile model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

In [67]:
# Fit the model
model_deep = nn.fit(X, y, validation_split=0.3, epochs=50)

Epoch 1/50
5/5 [==============================] - 2s 90ms/step - loss: 0.8895 - mse: 0.8895 - val_loss: 1.8627 - val_mse: 1.8627
Epoch 2/50
5/5 [==============================] - 0s 19ms/step - loss: 0.8790 - mse: 0.8790 - val_loss: 1.8586 - val_mse: 1.8586
Epoch 3/50
5/5 [==============================] - 0s 14ms/step - loss: 0.8687 - mse: 0.8687 - val_loss: 1.8546 - val_mse: 1.8546
Epoch 4/50
5/5 [==============================] - 0s 31ms/step - loss: 0.8591 - mse: 0.8591 - val_loss: 1.8508 - val_mse: 1.8508
Epoch 5/50
5/5 [==============================] - 0s 29ms/step - loss: 0.8495 - mse: 0.8495 - val_loss: 1.8473 - val_mse: 1.8473
Epoch 6/50
5/5 [==============================] - 0s 31ms/step - loss: 0.8400 - mse: 0.8400 - val_loss: 1.8438 - val_mse: 1.8438
Epoch 7/50
5/5 [==============================] - 0s 30ms/step - loss: 0.8302 - mse: 0.8302 - val_loss: 1.8404 - val_mse: 1.8404
Epoch 8/50
5/5 [==============================] - 0s 28ms/step - loss: 0.8216 - mse: 0.8216 - val

In [1]:
import matplotlib.pyplot as plt

### Train vs test for deep net
###plt.plot(model_deep.history["loss"])
###plt.plot(model_deep.history["val_loss"])
###plt.title("loss_function - Training Vs. Validation - 2 hidden layers")
###plt.legend(["train", "test"])
###plt.show()

In [2]:
# Plot training & validation accuracy values
###plt.plot(model_deep.history['accuracy'])
###plt.plot(model_deep.history['val_accuracy'])
###plt.title('Model accuracy')
###plt.xlabel('Epoch')
###plt.ylabel('Accuracy')
###plt.legend(['Train', 'Validation'], loc='upper left')
###plt.show()

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [25]:
# Making predictions using the testing data
#predictions = model_deep.predict(X_test_scaled)

In [28]:

# Calculating the confusion matrix
#cm = confusion_matrix(y_test, predictions)
#cm_df = pd.DataFrame(
   # cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]#

#)

# Calculating the accuracy score
#acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
#print("Confusion Matrix")
#display(cm_df)
#print(f"Accuracy Score : {acc_score}")
#print("Classification Report")
#print(classification_report(y_test, predictions))
 